In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
class color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    B = '\033[97m'
    BOLD = '\033[1m'
    DARKCYAN = '\033[36m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'  

In [3]:
data = pd.read_csv('laliga.csv')
data.head()

,id,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,timestamp
0,1,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970,21938400
1,2,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970,21938400
2,3,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970,21938400
3,4,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970,22024800
4,5,1970-71,1,1,Elche,Granada,1,1,13/09/1970,22024800


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37147 entries, 0 to 37146
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            37147 non-null  int64 
 1   season        37147 non-null  object
 2   division      37147 non-null  int64 
 3   round         37147 non-null  int64 
 4   localTeam     37147 non-null  object
 5   visitorTeam   37147 non-null  object
 6   localGoals    37147 non-null  int64 
 7   visitorGoals  37147 non-null  int64 
 8   date          37147 non-null  object
 9   timestamp     37147 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 2.8+ MB


In [5]:
data.isna().sum()

id              0
season          0
division        0
round           0
localTeam       0
visitorTeam     0
localGoals      0
visitorGoals    0
date            0
timestamp       0
dtype: int64

In [6]:
#View the duplicate dataset if any
d = data.duplicated()
#data[d]

In [7]:
shape = data.shape
dup = data.duplicated().sum()
print('Number of Duplicates in the Entire Dataset =', dup)
if dup >0:
    data.drop_duplicates(inplace=True)
    print('Number of Duplicates Dropped =',dup)
elif dup == 0:
    print('Duplicates are Nonexistent in the Dataset')
print('New Data Shape after Dropping Duplicates =',data.shape,'\n Old Data Shape =',shape)

Number of Duplicates in the Entire Dataset = 0
Duplicates are Nonexistent in the Dataset
New Data Shape after Dropping Duplicates = (37147, 10) 
 Old Data Shape = (37147, 10)


In [8]:
#Calculating the table
data['homeWin'] = (data['localGoals'] > data['visitorGoals'])*1 
data['homeLoss'] = (data['localGoals'] < data['visitorGoals'])*1
data['awayWin'] = (data['visitorGoals'] > data['localGoals'])*1 
data['awayLoss'] = (data['visitorGoals'] < data['localGoals'])*1
data['draw'] = (data['localGoals'] == data['visitorGoals'])*1 
data['homePoints'] = data['homeWin']*3 
data['awayPoints'] = data['awayWin']*3

In [9]:
# LT = Local Team; AT = Away Team; LTAT = Local and Away teams mereged as one
LT = data.groupby(['season','localTeam','division'])[['homePoints','awayPoints','draw']].sum().reset_index()
AT = data.groupby(['season','visitorTeam'])[['homePoints','awayPoints','draw']].sum().reset_index()
LTAT = pd.merge(LT,AT, right_index=True, left_index=True)
#LTAT.head()

In [10]:
# Calculate Total Point
LTAT['TotalPoint'] = LTAT['homePoints_x'] + LTAT['awayPoints_y'] + LTAT['draw_x'] + LTAT['draw_y']
#LTAT.head()

In [11]:
# Rename the columns
LTAT.columns=['season', 'Team', 'division','homePoints', 'homeLossPoints', 'homedraw',
       'season2', 'visitorTeam', 'awayLossPoints', 'awayPoints', 'awaydraw',
       'TotalPoint']
#LTAT

In [12]:
# Drop duplicate columns
LTAT = LTAT.drop(['season2','visitorTeam'],axis=1)
#LTAT

In [13]:
# Arrange the season in ascending and the total point in descending orders
LTAT = LTAT.sort_values(['season', 'TotalPoint'], ascending=[True, False])#.reset_index()
#LTAT

In [14]:
LTAT.groupby(['season', 'Team']).sum().reset_index()

,season,Team,division,homePoints,homeLossPoints,homedraw,awayLossPoints,awayPoints,awaydraw,TotalPoint
0,1970-71,Atletico de Bilbao,1,33,3,3,24,9,4,49
1,1970-71,Atletico de Madrid,1,39,0,2,15,12,6,59
2,1970-71,Barcelona,1,33,9,1,9,24,4,62
3,1970-71,Betis,2,45,3,3,12,15,10,73
4,1970-71,Burgos,2,48,0,3,36,9,4,64
...,...,...,...,...,...,...,...,...,...,...
1923,2017-18,Tenerife,2,33,6,8,33,12,6,59
1924,2017-18,Valencia,1,39,9,3,18,27,4,73
1925,2017-18,Valladolid,2,42,12,3,27,15,7,67
1926,2017-18,Villarreal,1,33,15,3,24,21,4,61


In [15]:
# Slice by season and by division to get seasonal tables
LTAT[(LTAT['season']=='1970-71') &  (LTAT['division']==1)].reset_index(drop=True)

,season,Team,division,homePoints,homeLossPoints,homedraw,awayLossPoints,awayPoints,awaydraw,TotalPoint
0,1970-71,Barcelona,1,33,9,1,9,24,4,62
1,1970-71,Valencia,1,36,3,2,12,18,5,61
2,1970-71,Atletico de Madrid,1,39,0,2,15,12,6,59
3,1970-71,Real Madrid,1,30,6,3,12,21,4,58
4,1970-71,Celta de Vigo,1,36,0,3,30,9,2,50
5,1970-71,Atletico de Bilbao,1,33,3,3,24,9,4,49
6,1970-71,Sevilla,1,33,3,3,30,6,3,45
7,1970-71,Real Sociedad,1,27,9,3,24,3,6,39
8,1970-71,Granada,1,27,3,5,33,3,3,38
9,1970-71,CD Malaga,1,18,9,6,21,6,6,36


In [16]:
season = data['season'].unique()
division = data['division'].unique()

In [17]:
# Do a test run to see if you can generate the teams arranged with numbers from the best  to the worst teams every season. 
#This is very important to the success of the report
for s in season:
    for d in division:
        number=0
        print(color.BLUE+'\n',s,'Division',d,color.END)
        summary = LTAT[(LTAT['season']==s) & (LTAT['division']==d)]
        for t in summary['Team'].unique():
            number+=1
            print(number,t)


 1970-71 Division 1 
1 Barcelona
2 Valencia
3 Atletico de Madrid
4 Real Madrid
5 Celta de Vigo
6 Atletico de Bilbao
7 Sevilla
8 Real Sociedad
9 Granada
10 CD Malaga
11 Sporting de Gijon
12 Espanol
13 Sabadell
14 Las Palmas
15 Elche
16 Zaragoza

 1970-71 Division 2 
1 Betis
2 Burgos
3 Deportivo
4 Rayo Vallecano
5 Cordoba
6 Castellon
7 Racing de Ferrol
8 Pontevedra
9 Sant Andreu
10 Mallorca
11 Santander
12 Hercules
13 Cadiz
14 Oviedo
15 Langreo
16 Logrones
17 Villarreal
18 Calvo Sotelo
19 Ontinyent
20 Moscardo

 1971-72 Division 1 
1 Real Madrid
2 Valencia
3 Barcelona
4 Atletico de Madrid
5 Las Palmas
6 Granada
7 Atletico de Bilbao
8 Real Sociedad
9 CD Malaga
10 Espanol
11 Sporting de Gijon
12 Celta de Vigo
13 Deportivo
14 Betis
15 Burgos
16 Sevilla
17 Cordoba
18 Sabadell

 1971-72 Division 2 
1 Oviedo
2 Castellon
3 Zaragoza
4 Elche
5 Cultural Leonesa
6 Logrones
7 Valladolid
8 Rayo Vallecano
9 Sant Andreu
10 Tenerife
11 Pontevedra
12 Cadiz
13 Santander
14 Hercules
15 Mallorca
16 Mestall

8 Compostela
9 Sabadell
10 Santander
11 Castellon
12 Palamos
13 Bilbao Athletic
14 Eibar
15 Murcia
16 Castilla
17 Sestao
18 CD Malaga
19 Aviles
20 Las Palmas

 1992-93 Division 1 
1 Barcelona
2 Real Madrid
3 Deportivo
4 Valencia
5 Sevilla
6 Atletico de Madrid
7 Tenerife
8 Atletico de Bilbao
9 Real Sociedad
10 Osasuna
11 Zaragoza
12 Sporting de Gijon
13 Logrones
14 Celta de Vigo
15 Oviedo
16 Albacete
17 Rayo Vallecano
18 Espanol
19 Cadiz
20 Real Burgos

 1992-93 Division 2 
1 Lleida
2 Santander
3 Valladolid
4 Mallorca
5 Atletico Marbella
6 Betis
7 Castilla
8 Barcelona Atletic
9 Merida
10 Badajoz
11 Castellon
12 Villarreal
13 Compostela
14 Palamos
15 Figueres
16 Bilbao Athletic
17 Eibar
18 Lugo
19 Sabadell
20 Sestao

 1993-94 Division 1 
1 Barcelona
2 Deportivo
3 Zaragoza
4 Real Madrid
5 Atletico de Bilbao
6 Sevilla
7 Valencia
8 Santander
9 Tenerife
10 Sporting de Gijon
11 Oviedo
12 Atletico de Madrid
13 Real Sociedad
14 Albacete
15 Celta de Vigo
16 Logrones
17 Rayo Vallecano
18 Valladol

1 Valladolid
2 Almeria
3 Murcia
4 Ciudad de Murcia
5 Cadiz
6 Albacete
7 Tenerife
8 Elche
9 Numancia
10 Poli Ejido
11 Xerez
12 Salamanca
13 Castellon
14 Sporting de Gijon
15 Malaga
16 Hercules
17 Alaves
18 Las Palmas
19 Castilla
20 Ponferradina
21 Lorca Deportiva
22 Vecindario

 2007-08 Division 1 
1 Real Madrid
2 Villarreal
3 Barcelona
4 Atletico de Madrid
5 Sevilla
6 Santander
7 Mallorca
8 Almeria
9 Deportivo
10 Valencia
11 Atletico de Bilbao
12 Espanol
13 Betis
14 Getafe
15 Valladolid
16 Recreativo de Huelva
17 Osasuna
18 Zaragoza
19 Murcia
20 Levante

 2007-08 Division 2 
1 Numancia
2 Malaga
3 Sporting de Gijon
4 Real Sociedad
5 Castellon
6 Hercules
7 Las Palmas
8 Salamanca
9 Sevilla Atletico
10 Elche
11 Tenerife
12 Albacete
13 Celta de Vigo
14 Eibar
15 Gimnastic de Tarragona
16 Xerez
17 Alaves
18 Cordoba
19 Racing de Ferrol
20 Cadiz
21 Granada 74
22 Poli Ejido

 2008-09 Division 1 
1 Barcelona
2 Real Madrid
3 Sevilla
4 Atletico de Madrid
5 Villarreal
6 Valencia
7 Deportivo
8 Malaga

# THE REPORT


In [18]:
print(color.CYAN+'Where: MP=Matches Played; GF=Goals For; GA=Goals Against; W=Win; GD=Goals Difference, D=Draw; TP=Total Points','LP=Loss Points'+color.END)
print(color.CYAN+'STATs: WR=Win Rate; LR=Loss Rate; DR=Draw Rate; GPG=Goals Per Game; GAPG=Goals Against Per Game; TPR=Total Points Ratio; TLPR= Total Loss Points Ratio'+color.END)
print()
for s in season:
    for d in division:
        number=0
        print(color.GREEN+color.BOLD+color.UNDERLINE+'\n\nDetails of Season',s,'|','Division',d,'',color.END)
        summary = LTAT[(LTAT['season']==s) & (LTAT['division']==d)]
        for t in summary['Team'].unique():
            number+=1
            k = data[(data['season']==s) & (data['division']==d)]#.sort_values('localTeam') 
           
            
            # Setup for calculations
            lt = k[(k['localTeam']) ==t]
            at = k[(k['visitorTeam']) ==t]
            home_goals1 = lt['localGoals'].sum()
            away_goals1 = at['visitorGoals'].sum()
            home_goals2 = lt['visitorGoals'].sum()
            away_goals2 = at['localGoals'].sum()
            homewin1 = lt['homeWin'].sum()
            awaywin1 = at['awayWin'].sum()
            homewin2 = lt['awayWin'].sum()
            awaywin2 = at['homeWin'].sum()
            draw1 = lt['draw'].sum() 
            draw2 = at['draw'].sum()
            totalPointH = homewin1*3 + draw1
            totalPointA = awaywin1*3 + draw2
            
            
            # Details of Season: Calculations
            matchPlayed = (len(lt)*2)
            goalsFor = home_goals1+away_goals1
            goalsAgainst = home_goals2+away_goals2
            win = homewin1+awaywin1
            lost = homewin2+awaywin2
            drawn = draw1+draw2
            maxPoints = matchPlayed*3
            totalPoint = win*3 + drawn
            lossPoint = maxPoints - totalPoint
            
            
            #Performance Statistics Calculations
            winRate = round((win/matchPlayed)*100,2)
            lossRate = round((lost/matchPlayed)*100,2)
            drawRate = round((drawn/matchPlayed)*100,2)
            gForPerGame = round((goalsFor/matchPlayed),2)
            gAgainstPerGame = round((goalsAgainst/matchPlayed),2)
            pointsForRatio = round((totalPoint/maxPoints)*100,2)
            lossPointsRatio = round((lossPoint/maxPoints)*100,2)
            
            
            #specify your search prameters here
            
            
            #Details of Season in Numbers
            print(color.BOLD+color.BLUE+'\n',number,':',t,color.END,
                    '\n',color.B+color.BOLD+'Performance(Home + Away)',color.END,
                    '\n','MP =',matchPlayed,';',
                    'Win =',win,';',
                    'Loss =',lost,';',
                    'Drawn =',drawn,';',
                    'GF =',goalsFor, 
                    ';','GA =',goalsAgainst,
                     ';','GD =',(goalsFor-goalsAgainst), 
                    ';','TP =',color.UNDERLINE+color.BOLD+color.GREEN,totalPoint,''+color.END,
                    ';','LP =',lossPoint)
            
            #Home Performances
            print(
                    color.PURPLE+color.BOLD+'','Home Performances:'+color.END,
                    '\n','MP =',len(lt),';',
                    'Win =',homewin1,';',
                    'Loss =',homewin2,';',
                    'Drawn =',draw1, ';',
                    'GF =',home_goals1, 
                    ';','GA =',home_goals2,
                     ';','GD =',(home_goals1-home_goals2),
                    ';','TP =',color.BOLD+color.GREEN+'',totalPointH,''+color.END)
            
            #Away Performances
            print(
                    color.YELLOW+color.BOLD+'','Away Performances:'+color.END,
                    '\n','MP =',len(lt),';',
                    'Win =',awaywin1,';',
                    'Loss =',awaywin2,';',
                    'Drawn =',draw2,';',
                    'GF =',away_goals1,
                    ';','GA =',away_goals2,
                     ';','GD =',(away_goals1-away_goals2), 
                    ';','TP =',color.BOLD+color.GREEN+'',totalPointA,''+color.END)
            
            #Performance Statistics in Numbers
            print(color.CYAN+color.BOLD+'','Overall Performance Statistics'+color.END,
                    '\n','WR =',winRate,'%',';',
                    'LR =',lossRate,'%', 
                    ';','DR =',drawRate,'%', 
                    ';','GPG =',gForPerGame,
                    ';','GAPG =',gAgainstPerGame,
                    ';','TPR =',pointsForRatio,'%',
                    ';','TLPR =',lossPointsRatio,'%'+color.END)
        
        #Season's Final Positions
        summary = LTAT[(LTAT['season']==s) & (LTAT['division']==d)]
        team2 = summary['Team'].unique()
        number2=0
        print(color.BLUE+color.BOLD+'\nFinal Positions for the Season',s, '| Division',d,''+color.END)
        for t in team2:
            position = summary[(summary['Team']) ==t]
            p = position['TotalPoint'].unique()
            number2+=1
            #for d in division:
            if number2 == 1:
                print(number2,color.PURPLE+color.BOLD+'',t,'winner with',*p,'Points'+color.END) 
            else:
                print(number2,'',t,'-',*p,'Points')
        
        
        #Champions League and Promotion Places
        print(color.BLUE+color.BOLD+'\nChampion League Places'+color.END) if d == 1 else ''
        print(color.BLUE+color.BOLD+'\nPromoted to Division 1'+color.END) if d == 2 else ''
        divi2 = summary['division'].unique()
        number3=0
        for t in team2:
            for d in divi2:
                position = summary[(summary['Team']) ==t]
                p = position['TotalPoint'].unique()
                number3+=1
                print(number3,'',t,'-',*p,'Points') if number3<=4 and d ==1 else''
                print(number3,'',t,'-',*p,'Points') if number3<=3 and d ==2 else ''
        
        
        #Relegation Places
        print(color.BLUE+color.BOLD+'\nRelegated To Division 3'+color.END) if d == 2 else''
        print(color.BLUE+color.BOLD+'\nRelegated To Division 2'+color.END) if d == 1 else''
        summary = LTAT[(LTAT['season']==s) & (LTAT['division']==d)].tail(3)
        team2 = summary['Team'].unique()
        for t in team2:
            position = summary[(summary['Team']) ==t]
            p = position['TotalPoint'].unique()
            print('**',t,'-',*p,'Points')
        
        #Season's Summary
        tg = k['localGoals'].sum() + k['visitorGoals'].sum()
        local = data[(data['season']== s) & (data['division']==d)]
        away = data[(data['season']== s) & (data['division']==d)]
        localG = local['localGoals'].sum() + local['visitorGoals'].sum()
        tw = k['homeWin'].sum() + k['awayWin'].sum()
        td = k['draw'].sum()
        GPG = round(tg/(len(k)),2)
        DR = round(td/(len(k)),2)
        WinorLossR =round(tw/(len(k)),2)
        print(color.BLUE+color.BOLD+'\n',s,'Division',d,'Aggregate Summary:',color.END,
                '\n','Total Games =',len(k), 
                ';','Total Goals =',tg,
                ';','Total Wins =',tw,
                ';','Total Draws =',td,
                ';','GPG =',GPG,
                ';','WinORLossRatio =',WinorLossR,
                ';','DR =',DR)
        print()

Where: MP=Matches Played; GF=Goals For; GA=Goals Against; W=Win; GD=Goals Difference, D=Draw; TP=Total Points LP=Loss Points
STATs: WR=Win Rate; LR=Loss Rate; DR=Draw Rate; GPG=Goals Per Game; GAPG=Goals Against Per Game; TPR=Total Points Ratio; TLPR= Total Loss Points Ratio



Details of Season 1970-71 | Division 1  

 1 : Barcelona  
 Performance(Home + Away)  
 MP = 30 ; Win = 19 ; Loss = 6 ; Drawn = 5 ; GF = 50 ; GA = 22 ; GD = 28 ; TP =  62  ; LP = 28
 Home Performances: 
 MP = 15 ; Win = 11 ; Loss = 3 ; Drawn = 1 ; GF = 32 ; GA = 11 ; GD = 21 ; TP =  34 
 Away Performances: 
 MP = 15 ; Win = 8 ; Loss = 3 ; Drawn = 4 ; GF = 18 ; GA = 11 ; GD = 7 ; TP =  28 
 Overall Performance Statistics 
 WR = 63.33 % ; LR = 20.0 % ; DR = 16.67 % ; GPG = 1.67 ; GAPG = 0.73 ; TPR = 68.89 % ; TLPR = 31.11 %

 2 : Valencia  
 Performance(Home + Away)  
 MP = 30 ; Win = 18 ; Loss = 5 ; Drawn = 7 ; GF = 41 ; GA = 19 ; GD = 22 ; TP =  61  ; LP = 29
 Home Performances: 
 MP = 15 ; Win = 12 ; Loss = 1 ;

 1970-71 Division 1 Aggregate Summary:  
 Total Games = 240 ; Total Goals = 543 ; Total Wins = 179 ; Total Draws = 61 ; GPG = 2.26 ; WinORLossRatio = 0.75 ; DR = 0.25



Details of Season 1970-71 | Division 2  

 1 : Betis  
 Performance(Home + Away)  
 MP = 38 ; Win = 20 ; Loss = 5 ; Drawn = 13 ; GF = 54 ; GA = 28 ; GD = 26 ; TP =  73  ; LP = 41
 Home Performances: 
 MP = 19 ; Win = 15 ; Loss = 1 ; Drawn = 3 ; GF = 38 ; GA = 10 ; GD = 28 ; TP =  48 
 Away Performances: 
 MP = 19 ; Win = 5 ; Loss = 4 ; Drawn = 10 ; GF = 16 ; GA = 18 ; GD = -2 ; TP =  25 
 Overall Performance Statistics 
 WR = 52.63 % ; LR = 13.16 % ; DR = 34.21 % ; GPG = 1.42 ; GAPG = 0.74 ; TPR = 64.04 % ; TLPR = 35.96 %

 2 : Burgos  
 Performance(Home + Away)  
 MP = 38 ; Win = 19 ; Loss = 12 ; Drawn = 7 ; GF = 46 ; GA = 30 ; GD = 16 ; TP =  64  ; LP = 50
 Home Performances: 
 MP = 19 ; Win = 16 ; Loss = 0 ; Drawn = 3 ; GF = 38 ; GA = 7 ; GD = 31 ; TP =  51 
 Away Performances: 
 MP = 19 ; Win = 3 ; Loss = 12 ; Draw

 Overall Performance Statistics 
 WR = 26.32 % ; LR = 39.47 % ; DR = 34.21 % ; GPG = 0.92 ; GAPG = 1.0 ; TPR = 37.72 % ; TLPR = 62.28 %

 17 : Villarreal  
 Performance(Home + Away)  
 MP = 38 ; Win = 11 ; Loss = 17 ; Drawn = 10 ; GF = 37 ; GA = 55 ; GD = -18 ; TP =  43  ; LP = 71
 Home Performances: 
 MP = 19 ; Win = 10 ; Loss = 4 ; Drawn = 5 ; GF = 25 ; GA = 17 ; GD = 8 ; TP =  35 
 Away Performances: 
 MP = 19 ; Win = 1 ; Loss = 13 ; Drawn = 5 ; GF = 12 ; GA = 38 ; GD = -26 ; TP =  8 
 Overall Performance Statistics 
 WR = 28.95 % ; LR = 44.74 % ; DR = 26.32 % ; GPG = 0.97 ; GAPG = 1.45 ; TPR = 37.72 % ; TLPR = 62.28 %

 18 : Calvo Sotelo  
 Performance(Home + Away)  
 MP = 38 ; Win = 11 ; Loss = 20 ; Drawn = 7 ; GF = 38 ; GA = 48 ; GD = -10 ; TP =  40  ; LP = 74
 Home Performances: 
 MP = 19 ; Win = 10 ; Loss = 4 ; Drawn = 5 ; GF = 32 ; GA = 16 ; GD = 16 ; TP =  35 
 Away Performances: 
 MP = 19 ; Win = 1 ; Loss = 16 ; Drawn = 2 ; GF = 6 ; GA = 32 ; GD = -26 ; TP =  5 
 Overall Per


 10 : Espanol  
 Performance(Home + Away)  
 MP = 34 ; Win = 12 ; Loss = 14 ; Drawn = 8 ; GF = 43 ; GA = 43 ; GD = 0 ; TP =  44  ; LP = 58
 Home Performances: 
 MP = 17 ; Win = 10 ; Loss = 3 ; Drawn = 4 ; GF = 31 ; GA = 12 ; GD = 19 ; TP =  34 
 Away Performances: 
 MP = 17 ; Win = 2 ; Loss = 11 ; Drawn = 4 ; GF = 12 ; GA = 31 ; GD = -19 ; TP =  10 
 Overall Performance Statistics 
 WR = 35.29 % ; LR = 41.18 % ; DR = 23.53 % ; GPG = 1.26 ; GAPG = 1.26 ; TPR = 43.14 % ; TLPR = 56.86 %

 11 : Sporting de Gijon  
 Performance(Home + Away)  
 MP = 34 ; Win = 12 ; Loss = 14 ; Drawn = 8 ; GF = 41 ; GA = 37 ; GD = 4 ; TP =  44  ; LP = 58
 Home Performances: 
 MP = 17 ; Win = 10 ; Loss = 3 ; Drawn = 4 ; GF = 29 ; GA = 11 ; GD = 18 ; TP =  34 
 Away Performances: 
 MP = 17 ; Win = 2 ; Loss = 11 ; Drawn = 4 ; GF = 12 ; GA = 26 ; GD = -14 ; TP =  10 
 Overall Performance Statistics 
 WR = 35.29 % ; LR = 41.18 % ; DR = 23.53 % ; GPG = 1.21 ; GAPG = 1.09 ; TPR = 43.14 % ; TLPR = 56.86 %

 12 : Cel


 8 : Rayo Vallecano  
 Performance(Home + Away)  
 MP = 38 ; Win = 14 ; Loss = 13 ; Drawn = 11 ; GF = 37 ; GA = 40 ; GD = -3 ; TP =  53  ; LP = 61
 Home Performances: 
 MP = 19 ; Win = 9 ; Loss = 5 ; Drawn = 5 ; GF = 19 ; GA = 15 ; GD = 4 ; TP =  32 
 Away Performances: 
 MP = 19 ; Win = 5 ; Loss = 8 ; Drawn = 6 ; GF = 18 ; GA = 25 ; GD = -7 ; TP =  21 
 Overall Performance Statistics 
 WR = 36.84 % ; LR = 34.21 % ; DR = 28.95 % ; GPG = 0.97 ; GAPG = 1.05 ; TPR = 46.49 % ; TLPR = 53.51 %

 9 : Sant Andreu  
 Performance(Home + Away)  
 MP = 38 ; Win = 15 ; Loss = 15 ; Drawn = 8 ; GF = 36 ; GA = 33 ; GD = 3 ; TP =  53  ; LP = 61
 Home Performances: 
 MP = 19 ; Win = 13 ; Loss = 2 ; Drawn = 4 ; GF = 31 ; GA = 11 ; GD = 20 ; TP =  43 
 Away Performances: 
 MP = 19 ; Win = 2 ; Loss = 13 ; Drawn = 4 ; GF = 5 ; GA = 22 ; GD = -17 ; TP =  10 
 Overall Performance Statistics 
 WR = 39.47 % ; LR = 39.47 % ; DR = 21.05 % ; GPG = 0.95 ; GAPG = 0.87 ; TPR = 46.49 % ; TLPR = 53.51 %

 10 : Tenerif

...

 MP = 19 ; Win = 3 ; Loss = 11 ; Drawn = 5 ; GF = 14 ; GA = 32 ; GD = -18 ; TP =  14 
 Overall Performance Statistics 
 WR = 21.05 % ; LR = 50.0 % ; DR = 28.95 % ; GPG = 0.95 ; GAPG = 1.45 ; TPR = 30.7 % ; TLPR = 69.3 %

 18 : Sporting de Gijon  
 Performance(Home + Away)  
 MP = 38 ; Win = 7 ; Loss = 21 ; Drawn = 10 ; GF = 42 ; GA = 72 ; GD = -30 ; TP =  31  ; LP = 83
 Home Performances: 
 MP = 19 ; Win = 5 ; Loss = 10 ; Drawn = 4 ; GF = 26 ; GA = 38 ; GD = -12 ; TP =  19 
 Away Performances: 
 MP = 19 ; Win = 2 ; Loss = 11 ; Drawn = 6 ; GF = 16 ; GA = 34 ; GD = -18 ; TP =  12 
 Overall Performance Statistics 
 WR = 18.42 % ; LR = 55.26 % ; DR = 26.32 % ; GPG = 1.11 ; GAPG = 1.89 ; TPR = 27.19 % ; TLPR = 72.81 %

 19 : Osasuna  
 Performance(Home + Away)  
 MP = 38 ; Win = 4 ; Loss = 24 ; Drawn = 10 ; GF = 40 ; GA = 94 ; GD = -54 ; TP =  22  ; LP = 92
 Home Performances: 
 MP = 19 ; Win = 2 ; Loss = 10 ; Drawn = 7 ; GF = 23 ; GA = 39 ; GD = -16 ; TP =  13 
 Away Performances: 
 MP = 1


 11 : Reus  
 Performance(Home + Away)  
 MP = 42 ; Win = 13 ; Loss = 13 ; Drawn = 16 ; GF = 31 ; GA = 29 ; GD = 2 ; TP =  55  ; LP = 71
 Home Performances: 
 MP = 21 ; Win = 7 ; Loss = 5 ; Drawn = 9 ; GF = 18 ; GA = 15 ; GD = 3 ; TP =  30 
 Away Performances: 
 MP = 21 ; Win = 6 ; Loss = 8 ; Drawn = 7 ; GF = 13 ; GA = 14 ; GD = -1 ; TP =  25 
 Overall Performance Statistics 
 WR = 30.95 % ; LR = 30.95 % ; DR = 38.1 % ; GPG = 0.74 ; GAPG = 0.69 ; TPR = 43.65 % ; TLPR = 56.35 %

 12 : Rayo Vallecano  
 Performance(Home + Away)  
 MP = 42 ; Win = 14 ; Loss = 17 ; Drawn = 11 ; GF = 44 ; GA = 44 ; GD = 0 ; TP =  53  ; LP = 73
 Home Performances: 
 MP = 21 ; Win = 10 ; Loss = 4 ; Drawn = 7 ; GF = 29 ; GA = 16 ; GD = 13 ; TP =  37 
 Away Performances: 
 MP = 21 ; Win = 4 ; Loss = 13 ; Drawn = 4 ; GF = 15 ; GA = 28 ; GD = -13 ; TP =  16 
 Overall Performance Statistics 
 WR = 33.33 % ; LR = 40.48 % ; DR = 26.19 % ; GPG = 1.05 ; GAPG = 1.05 ; TPR = 42.06 % ; TLPR = 57.94 %

 13 : Sevilla Atle


 4 : Valencia  
 Performance(Home + Away)  
 MP = 38 ; Win = 22 ; Loss = 9 ; Drawn = 7 ; GF = 65 ; GA = 38 ; GD = 27 ; TP =  73  ; LP = 41
 Home Performances: 
 MP = 19 ; Win = 13 ; Loss = 3 ; Drawn = 3 ; GF = 36 ; GA = 16 ; GD = 20 ; TP =  42 
 Away Performances: 
 MP = 19 ; Win = 9 ; Loss = 6 ; Drawn = 4 ; GF = 29 ; GA = 22 ; GD = 7 ; TP =  31 
 Overall Performance Statistics 
 WR = 57.89 % ; LR = 23.68 % ; DR = 18.42 % ; GPG = 1.71 ; GAPG = 1.0 ; TPR = 64.04 % ; TLPR = 35.96 %

 5 : Villarreal  
 Performance(Home + Away)  
 MP = 38 ; Win = 18 ; Loss = 13 ; Drawn = 7 ; GF = 57 ; GA = 50 ; GD = 7 ; TP =  61  ; LP = 53
 Home Performances: 
 MP = 19 ; Win = 11 ; Loss = 5 ; Drawn = 3 ; GF = 35 ; GA = 22 ; GD = 13 ; TP =  36 
 Away Performances: 
 MP = 19 ; Win = 7 ; Loss = 8 ; Drawn = 4 ; GF = 22 ; GA = 28 ; GD = -6 ; TP =  25 
 Overall Performance Statistics 
 WR = 47.37 % ; LR = 34.21 % ; DR = 18.42 % ; GPG = 1.5 ; GAPG = 1.32 ; TPR = 53.51 % ; TLPR = 46.49 %

 6 : Betis  
 Performanc

16  Atletico de Bilbao - 43 Points
17  Leganes - 43 Points
18  Deportivo - 29 Points
19  Las Palmas - 22 Points
20  Malaga - 20 Points

Champion League Places
1  Barcelona - 93 Points
2  Atletico de Madrid - 79 Points
3  Real Madrid - 76 Points
4  Valencia - 73 Points

Relegated To Division 2
** Deportivo - 29 Points
** Las Palmas - 22 Points
** Malaga - 20 Points

 2017-18 Division 1 Aggregate Summary:  
 Total Games = 380 ; Total Goals = 1024 ; Total Wins = 294 ; Total Draws = 86 ; GPG = 2.69 ; WinORLossRatio = 0.77 ; DR = 0.23



Details of Season 2017-18 | Division 2  

 1 : Rayo Vallecano  
 Performance(Home + Away)  
 MP = 42 ; Win = 21 ; Loss = 8 ; Drawn = 13 ; GF = 67 ; GA = 48 ; GD = 19 ; TP =  76  ; LP = 50
 Home Performances: 
 MP = 21 ; Win = 13 ; Loss = 3 ; Drawn = 5 ; GF = 41 ; GA = 23 ; GD = 18 ; TP =  44 
 Away Performances: 
 MP = 21 ; Win = 8 ; Loss = 5 ; Drawn = 8 ; GF = 26 ; GA = 25 ; GD = 1 ; TP =  32 
 Overall Performance Statistics 
 WR = 50.0 % ; LR = 19.05 % ; 

 MP = 21 ; Win = 10 ; Loss = 7 ; Drawn = 4 ; GF = 36 ; GA = 25 ; GD = 11 ; TP =  34 
 Away Performances: 
 MP = 21 ; Win = 5 ; Loss = 14 ; Drawn = 2 ; GF = 21 ; GA = 40 ; GD = -19 ; TP =  17 
 Overall Performance Statistics 
 WR = 35.71 % ; LR = 50.0 % ; DR = 14.29 % ; GPG = 1.36 ; GAPG = 1.55 ; TPR = 40.48 % ; TLPR = 59.52 %

 17 : Albacete  
 Performance(Home + Away)  
 MP = 42 ; Win = 11 ; Loss = 15 ; Drawn = 16 ; GF = 35 ; GA = 46 ; GD = -11 ; TP =  49  ; LP = 77
 Home Performances: 
 MP = 21 ; Win = 8 ; Loss = 6 ; Drawn = 7 ; GF = 19 ; GA = 17 ; GD = 2 ; TP =  31 
 Away Performances: 
 MP = 21 ; Win = 3 ; Loss = 9 ; Drawn = 9 ; GF = 16 ; GA = 29 ; GD = -13 ; TP =  18 
 Overall Performance Statistics 
 WR = 26.19 % ; LR = 35.71 % ; DR = 38.1 % ; GPG = 0.83 ; GAPG = 1.1 ; TPR = 38.89 % ; TLPR = 61.11 %

 18 : Almeria  
 Performance(Home + Away)  
 MP = 42 ; Win = 12 ; Loss = 18 ; Drawn = 12 ; GF = 38 ; GA = 45 ; GD = -7 ; TP =  48  ; LP = 78
 Home Performances: 
 MP = 21 ; Win = 9 ;